In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler, TensorBoard
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.applications import EfficientNetB7
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import time
from IPython.display import clear_output

# Check GPU availability
print("GPU Available:", tf.config.list_physical_devices('GPU'))

# Enable  mixed precision for better GPU performance
tf.keras.mixed_precision.set_global_policy('mixed_float16')

# Define dataset paths
train_dir = 'data/Testing'
test_dir = 'data/Training'

# Check if directories exist
print("Trainingu Directory:", os.path.exists(train_dir))
print("Testing Directory:", os.path.exists(test_dir))s

# Image dimensions and batch size
img_width, img_height = 224, 224
batch_size = 32

class TrainingVisualizer(tf.keras.callbacks.Callback):
    def __init__(self):
        super(TrainingVisualizer, self).__init__()e
        self.epoch_times = []
        self.training_logs = []
        self.start_time = time.time()

    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start_time = time.time()
        print(f"\nEpoch {epoch + 1} starting...")
        
    def on_epoch_end(self, epoch, logs=None):
        epoch_time = time.time() - self.epoch_start_time
        self.epoch_times.append(epoch_time)
        self.training_logs.append(logs)
        
        clear_output(wait=True)
        
        total_time = time.time() - self.start_time
        avg_epoch_time = np.mean(self.epoch_times)
        estimated_time_remaining = avg_epoch_time * (self.params['epochs'] - epoch - 1)
        
        print(f"\nEpoch {epoch + 1}/{self.params['epochs']}")
        print(f"Time taken for epoch: {epoch_time:.2f}s")
        print(f"Average epoch time: {avg_epoch_time:.2f}s")
        print(f"Total training time: {total_time/60:.2f}m")
        print(f"Estimated time remaining: {estimated_time_remaining/60:.2f}m")
        
        print("\nMetrics:")
        print(f"Loss: {logs['loss']:.4f}")
        print(f"Accuracy: {logs['accuracy']:.4f}")
        print(f"Validation Loss: {logs['val_loss']:.4f}")
        print(f"Validation Accuracy: {logs['val_accuracy']:.4f}")
        
        self.plot_progress(epoch)
    
    def plot_progress(self, epoch):
        plt.figure(figsize=(12, 4))
        
        plt.subplot(1, 2, 1)
        plt.plot([log['accuracy'] for log in self.training_logs], label='Training Accuracy')
        plt.plot([log['val_accuracy'] for log in self.training_logs], label='Validation Accuracy')
        plt.title('Model Accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend()
        
        plt.subplot(1, 2, 2)
        plt.plot([log['loss'] for log in self.training_logs], label='Training Loss')
        plt.plot([log['val_loss'] for log in self.training_logs], label='Validation Loss')
        plt.title('Model Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        
        plt.tight_layout()
        plt.show()

# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

# Create model
print("Creating model...")
base_model = EfficientNetB7(
    weights='imagenet',
    include_top=False,
    input_shape=(img_width, img_height, 3)
)

# Unfreeze the top 20 layers
for layer in base_model.layers[-20:]:
    layer.trainable = True

# Build the model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')
])

# Compile model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=['accuracy', Precision(), Recall()]
)

print("Model created and compiled successfully!")

# Create output directory
os.makedirs('model_output', exist_ok=True)

# Callbacks
checkpoint = ModelCheckpoint(
    'model_output/brain_tumor_modelv2.keras',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=5,
    min_lr=1e-6,
    verbose=1
)

def cosine_decay(epoch, lr):
    epochs = 20
    alpha = 0.0
    cosine_decay = 0.5 * (1 + np.cos(np.pi * epoch / epochs))
    decayed_lr = (1 - alpha) * cosine_decay + alpha
    return lr * decayed_lr

lr_scheduler = LearningRateScheduler(cosine_decay)
tensorboard = TensorBoard(log_dir='model_output/logs', histogram_freq=1)
visualizer = TrainingVisualizer()

# Train model
print("Starting training...")
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size,
    epochs=20,
    callbacks=[checkpoint, reduce_lr, lr_scheduler, tensorboard, visualizer]
)

# Evaluate model
print("\nEvaluating model...")
loss, accuracy, precision, recall = model.evaluate(test_generator)
print(f'\nFinal Metrics:')
print(f'Loss: {loss:.4f}')
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')

# Generate and print classification report
Y_pred = model.predict(test_generator)
y_pred = np.argmax(Y_pred, axis=1)
print('\nClassification Report:')
print(classification_report(
    test_generator.classes,
    y_pred,
    target_names=list(test_generator.class_indices.keys())
))